# Courses

A set of tables for handling teaching activities


In [15]:
#| default_exp Courses

In [16]:
#| export
#| hide
from biobib.table import Table, sh
from biobib.sheet import Sheet
from nbdev import show_doc
import pandas as pd
import copy

In [17]:
#| hide
from fastcore.test import *
from fastcore.test import *
from fastcore.basics import *
from fastcore.foundation import *

### Define a template for service activities

Here is a template we can use for service activities. It generates a longtable which can span multiple pages, and provides the year, role, and type of each service activity.


In [18]:
courses_biobib_template = r'''% UC Bio-bib Catalog Courses Table
% Created on \VAR{created}

\begin{longtable}{p{1cm}p{7cm}p{0.75cm}rrrrp{2.5cm}}
Qtr & Course & Class Type & IWC & Units & Hrs/Wk & Enrollment & ESCI/Written Evals Avail.\\
\hline 
\endfirsthead

\multicolumn{7}{c}%
{{\VAR{table_name} - continued from previous page }} \\ \\
Qtr & Course & Class Type & IWC & Units & Hrs/Wk & Enrollment & ESCI/Written Evals Avail.\\
\hline 
\endhead

\\
\multicolumn{7}{c}%
{{ \VAR{table_name} continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot


\BLOCK{ for course in items }
\VAR{course.QYR} & \VAR{course.Course}, \VAR{course.Title} & \VAR{course['Class Type']} & \VAR{course.IWC|tex_escape} & \VAR{course.Units|tex_escape} & \VAR{course['Hours per Week']|tex_escape} & \VAR{ course.Enrollment|tex_escape} & \VAR{course.ESCI|tex_escape}/\VAR{course.Evals|tex_escape} \BLOCK{if course.Evals} \VAR{course['ESCI Link']|href} \BLOCK{endif} \\ 
\BLOCK{ endfor } 
\end{longtable}
'''


In [53]:
courses_cv_template = r'''% CV Catalog Courses Table
% Created on \VAR{created}

\begin{longtable}{p{1cm}p{7cm}}
Year & Course \\
\hline 
\endfirsthead

\multicolumn{2}{l}%
{{\VAR{table_name} - continued from previous page }} \\ \\
Year & Course \\
\hline 
\endhead

\\
\multicolumn{2}{l}%
{{ \VAR{table_name} continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot

\BLOCK{ for course in items }
\BLOCK{ if course['Class Type'] == 'Lec' }
\VAR{course.Year} & \VAR{course.Course}, \VAR{course.Title} \\ 
\BLOCK{ endif }
\BLOCK{ endfor } 
\end{longtable}
'''



In [19]:
class Courses(Table):
    """ 
    A Courses class of Table for use in generating biobibs and CVs 
    """
    def __init__(
            self,
            sheet:Sheet=None, # Google sheet object
            worksheet:str='Courses',   # worksheet name in google sheet
            csv_file:str=None, # optional csv file (if using csv files) 
            table_name:str=None, # Will be determined from category.
            cumulative:bool=False, # Is this table cumulative? 
            template=courses_biobib_template # template content
    ):
        filters = {
            'href': self.href
        }
        
        super().__init__(
            sheet=sheet,
            worksheet=worksheet, 
            table_name=table_name,
            csv_file=csv_file, 
            template=template,
            filters=filters)
        self.cumulative = cumulative
        self.df = self.clean_df()

### Provide a data cleaning function for the courses table



In [27]:
#| export

@patch
def clean_df(self:Courses,
             sort_by:list=['Year', 'Q', 'Course'], # variable to sort by
             ascending:list=[True, True, True], # ascending?
            )->pd.DataFrame:  # cleaned dataframe
    """
    Clean the Service table.
    
    """
    df = copy.deepcopy(self.df)
    if not self.cumulative:
        df = self.clean_cumulative(df)
    df = Table.table_clean_df(self)
    df = df.sort_values(by=sort_by, ascending=ascending)
    return df

In [28]:
sh.get_dataframe

<bound method Sheet.get_dataframe of <biobib.sheet.Sheet object at 0x176b5d1b0>>

In [35]:
courses_table = Courses(sheet=sh,worksheet='Courses',table_name='Courses',template=courses_biobib_template)

In [36]:
test_eq(courses_table.table_name,'Courses')
test_eq(list(courses_table.df.columns),
        ['Year', 'Q', 'Quarter', 'QYR', 'Course', 'Title', 
         'Class Type', 'IWC', 'Units', 'Hours per Week', 
         'Enrollment', 'ESCI', 'ESCI Link', 
         'Evals', 'Eval', 'Q1', 'Q2']
        )

In [37]:
print(courses_table.render_template())

% UC Bio-bib Catalog Courses Table
% Created on 2023-06-29 16:41

\begin{longtable}{p{1cm}p{7cm}p{0.75cm}rrrrp{2.5cm}}
Qtr & Course & Class Type & IWC & Units & Hrs/Wk & Enrollment & ESCI/Written Evals Avail.\\
\hline 
\endfirsthead

\multicolumn{7}{c}%
{{Courses - continued from previous page }} \\ \\
Qtr & Course & Class Type & IWC & Units & Hrs/Wk & Enrollment & ESCI/Written Evals Avail.\\
\hline 
\endhead

\\
\multicolumn{7}{c}%
{{ Courses continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot


F18 & Credit, Geography IWC Credit for ERI Directorship & - & 0.75 &  &  &  & /  \\ 
F18 & Credit, Bren IWC Credit for ERI Directorship & - & 0.88 &  &  &  & /  \\ 
F18 & ESM 203, Earth System Science & Lec & 1 & 4 & 1 & 90 & Yes/Yes  \href{https://ucsb.box.com/s/vvhpxerul6hbzum3fvmwigzd3y8yset6}{[link]}  \\ 
F18 & ESM 401B, Group Project - B & Dis & 0.29 & 4 & 1 & 5 & No/No  \href{}{[link]}  \\ 
F18 & GEOG 199RA, Independent Research Assistant & Tut & 0 &  & 1 & 1 & No/No  \href

In [54]:
cv_table = Courses(sheet=sh,worksheet='Courses',table_name='Courses',template=courses_cv_template)

In [55]:
print(cv_table.render_template())

% CV Catalog Courses Table
% Created on 2023-06-29 16:55

\begin{longtable}{p{1cm}p{7cm}}
Year & Course \\
\hline 
\endfirsthead

\multicolumn{2}{l}%
{{Courses - continued from previous page }} \\ \\
Year & Course \\
\hline 
\endhead

\\
\multicolumn{2}{l}%
{{ Courses continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot

2018 & ESM 203, Earth System Science \\ 
2019 & ESM 401A, Group Project - A \\ 
2019 & G136, Water, Energy, and Ecosystems \\ 
2019 & ESM 203, Earth System Science \\ 
2020 & ESM 401A, Group Project - A \\ 
2020 & GEOG 136, Water, Energy, and Ecosystems \\ 
2020 & ESM 203, Earth System Science \\ 
2021 & ESM 401A, Group Project - A \\ 
 
\end{longtable}


In [38]:
#| hide
import nbdev; nbdev.nbdev_export()